# Time-Lapse Microscopy
---

<img 
    src="./assets/timelapse_header.png" 
    alt="Time-Lapse Microscopy header" 
    align="center" 
    style="border: 2px solid #ccc; border-radius: 8px; padding: 5px; width: 100%; box-shadow: 0px 4px 8px rgba(0,0,0,0.1);">


In [ ]:
from IPython.display import HTML
HTML("""
<div style="display: flex; justify-content: center; padding: 10px;">
    <iframe width="560" height="315" src="https://www.youtube.com/embed/Qa-wrIdMYH0?si=KDzApOEt2e4ROu-l" title="YouTube video player" frameborder="0" allow="accelerometer; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>
</div>
""")

---

## Overview

This workflow demonstrates the display of time-lapse microscopy in neuroscience. Each frame in the image-stack dataset corresponds to a concurrent time sample, typically intended to capture a dynamic process of living cells.

For example, a dynamic process of interest could be [neural action potentials](https://en.wikipedia.org/wiki/Action_potential), and the data might come from a [miniature microscope](http://miniscope.org/) (**see image in this notebook's header**) that captures the change in fluorescence of special proteins caused by electrochemical fluctuations indicative of neuronal activity. These video-like datasets often contain many more frames in the 'Time' dimension compared to the number of pixels in the height or width of each frame.


### App Versions
We will build three different visualization approaches to cater to different use cases:

1. **Basic Viewer:** A one-line application using [hvPlot](https://hvplot.holoviz.org/) for quick visualization of the image stack.
2. **Intermediate Viewer with Side Views:** Uses [HoloViews](https://holoviews.org/) for additional interactive elements, scalebars, and linked side views, aiding better navigation of the image stack and identification of regions of interest.
3. **Advanced Viewer with Annotations and Linked Timeseries:** Building from the intermediate `HoloViews` viewer, this version adds annotation capabilities using [HoloNote](https://holonote.holoviz.org/), allowing for interactive spatial annotations with linked timeseries.

These applications are designed to handle large datasets efficiently, leveraging tools like `Xarray`, `Dask`, and `Zarr` for scalable data management.

### Use Cases

- **Basic Viewer:** Ideal for quick inspections and preliminary analyses of image stacks.
- **Intermediate Viewer:** Provides enhanced navigation through the data with side views, useful for identifying regions of interest.
- **Advanced Viewer:** Enables detailed analysis with spatial annotations and timeseries plots, suitable for in-depth studies of dynamic processes in microscopy data.

## Prerequisites

| Topic | Type | Notes |
| --- | --- | --- |
| [Xarray Tutorial](https://tutorial.xarray.dev/overview/xarray-in-45-min) | Prerequisite | Essential introduction to working with `xarray` data |

---

## Imports and Configuration


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import holoviews as hv
from holoviews.operation.datashader import rasterize
import hvplot.xarray # noqa
import panel as pn
import fsspec

pn.extension('tabulator')
hv.extension('bokeh')

## Loading and Inspecting the Data

We'll be working with a sample dataset of time-lapse microscopy images. The dataset is stored in Zarr format, which is optimized for chunked, compressed, and scalable storage.

In [ ]:
DATA_URL = 'https://datasets.holoviz.org/miniscope/v1/real_miniscope_uint8.zarr/'
DATA_DIR = Path('./data')
DATA_FILENAME = Path(DATA_URL).name
DATA_PATH = DATA_DIR / DATA_FILENAME

print(f'Local Data Path: {DATA_PATH}')

Let's download the dataset (if it wasn't already) so we can have a local copy and avoid any network delays. However, this workflow should also work if the dataset stays remote (thanks to Xarray, Zarr, Dask, and other scalability-providing tools), such as when it's too large to reasonably download in its entirety.

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Note</p>
    If you are viewing this notebook as a result of using the `anaconda-project run` command, the data has already been ingested, as configured in the associated yaml file. Running the following cell should find that data and skip any further download.
</div>

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Warning</p>
    If the data was not previously ingested with `anaconda-project`, the following cell will download ~300 MB the first time it is run.
</div>

In [ ]:
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Download the data if it doesn't exist
if not DATA_PATH.exists():
    print(f'Downloading data to: {DATA_PATH}')
    ds_remote = xr.open_dataset(
        fsspec.get_mapper(DATA_URL), engine='zarr', chunks={}
    )
    ds_remote.to_zarr(str(DATA_PATH))  # Save locally
    print(f'Dataset downloaded to: {DATA_PATH}')
else:
    print(f'Data exists at: {DATA_PATH}')

Now, let's load the dataset using `xarray`, specifying chunks for efficient data handling with `Dask`.


In [ ]:
# Open the dataset from the local copy
ds = xr.open_dataset(
    DATA_PATH.as_posix(),
    engine='zarr',
    chunks={'frame': 400, 'height': -1, 'width': -1}  # Chunk by frames
)

# Access the variable 'varr_ref' which contains the image data
da = ds['varr_ref']
da

The dataset `da` is a 3D array with dimensions `(frame, height, width)`. Each frame corresponds to a time point in the image stack.


---

## App V1: Basic Viewer with hvPlot

Our first application is a simple viewer using `hvPlot`, which allows for quick visualization of the image stack with minimal code.


In [ ]:
hvplot_app = da.hvplot.image(
    groupby="frame",
    title='hvPlot App',
    cmap='viridis',
    clim=(0, 20),
    aspect=da.sizes['width'] / da.sizes['height'],
    widget_location='bottom',
)
hvplot_app

As you can see, this creates an interactive image viewer where you can navigate through frames using a slider widget. Not much more needs to be said about that; it's simple and effective in a pinch! 

To easily enrich and extend this simple app, we do things like add a maximum-projection image so we can see the maximum fluorescence per pixel and visually locate the potential neurons in two-dimensions.

In [ ]:
max_proj = da.max('frame').compute().astype(np.float32)
img_max_proj = max_proj.hvplot.image(
    title='Max Over Time',
    cmap="magma",
    clim=(0,20),
    aspect=da.sizes['width'] / da.sizes['height']
)

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Note</p>
    Displaying the same interactive views or components across different cells in a notebook may lead to unintended consequences, as they are intentionally linked by default and can interfere. For this reason, many of the previews of the incremental steps in this tutorial are static previews.
</div>

In [ ]:
# pn.Row(hvplot_app, img_max_proj)

<img 
    src="./assets/basic_max_app.png"  
    alt="Static Preview Basic app w Max" 
    align="right" 
    width="70%">

**Here's a static snapshot of what the previous cell produces - the basic app with a max projection over time. 👉**

<div style="clear: both;"></div>

This was a quick way to see one frame at a time! But it looks like there are a lot of fluorescent blobs (candidate neurons) in the 'Max Over Time' image and now we want a quick way to visually locate and navigate to the relevant frames in the image stack.

## App V2: Intermediate Viewer with Side Views

As our data array is a three-dimensional volume, let's create a more advanced application using `HoloViews` in place of `hvPlot` to handle the added complexity, and `Panel` for more control over the layout and interactive links.

This more advanced app builds on the previous one with added functionality, such as.

1. **Side Views**: Aggregated side views for display over 'deep' time dimension.
2. **Synchronized Frame Indicators**: Frame markers on the Side Views synchronized with the playback and x,y range of the main image stack view.
4. **Slider Overlay Opacity**: Slider widget to adjust opacity of max-over-time overlay for direct comparison and a tighter layout.
5. **Scale Bar**: A dynamic and customizable visual reference for spatial scale.
3. **Continuous Playback**: Player widget for continuous playback, along with controls for step-by-step examination of the image stack.

### Main Image and Playback Controls


<img 
    src="./assets/int_app_main_w_player.png"
    alt="Static Preview Intermediate app Main w Player" 
    align="right"
    width=70%
    >

**Here's a static snapshot of the main view and frame player that we'll make now. 👉**


<div style="clear: both;"></div>

First, we'll define a function to create the main image view for a given frame.

In [ ]:
def plot_image(frame):
    return hv.Image(da.sel(frame=frame), kdims=["width", "height"]).opts(
        title=f'Frame = {frame}',
        frame_height=da.sizes['height'],
        frame_width=da.sizes['width'],
        cmap='Viridis',
        clim=(0, 20),
        colorbar=True,
        tools=['hover', 'crosshair'],
        toolbar='right',
        apply_hard_bounds=True,
        scalebar=True,
        scalebar_unit=("µm", "m"),  # Each pixel is about 1 µm in this dataset
        scalebar_opts={
            'background_fill_alpha': 0.5,
            'border_line_color': None,
            'bar_length': 0.10,
        }
    )

We'll also create a `Player` widget to control frame playback.

In [ ]:
frame_player = pn.widgets.Player(
    length=len(da.coords['frame']),
    interval=100,  # Inter-frame-interval in milliseconds
    value=20,     # Arbitrary starting frame
    show_loop_controls=False,
    align='center',
    scale_buttons=0.9,
    sizing_mode='stretch_width',
    show_value=True,
    value_align='center',
    visible_buttons=['slower', 'previous', 'pause', 'play', 'next', 'faster'],
)

We will `bind` (see [relevant Panel docs](https://panel.holoviz.org/explanation/api/reactivity.html)) the main frame-wise view to the player widget. Using [`DynamicMap`](https://holoviews.org/reference/containers/bokeh/DynamicMap.html), we ensure that only the plot contents are updated, maintaining the zoom level and other plot settings across updates. Additionally, by binding to [`value_throttled`](https://panel.holoviz.org/explanation/components/components_overview.html#throttling), we update the frame only when the user releases the slider, which improves performance by avoiding unnecessary updates:

In [ ]:
main_view = hv.DynamicMap(pn.bind(plot_image, frame_player.param.value_throttled))

### Maximum Projection Overlay

<img 
    src="./assets/int_app_max_w_slider.png"  
    alt="Static Preview Intermediate App Max Projection with Slider" 
    align="right" 
    width="70%">

**Here's a static snapshot of the max projection with a linked opacity slider that we'll make now. 👉**

<div style="clear: both;"></div>

We now create a max time-projected image and a slider widget to adjust the transparency of this overlay. As before, the max projection helps in identifying areas of interest by showing the maximum value over time for each pixel. We'll use a fast [`jslink`](https://panel.holoviz.org/how_to/links/jslinks.html) approach to link to the slider to the opacity parameter of the image since this is a simple visual property update.

In [ ]:
# Compute the maximum projection over time
max_proj_time = da.max('frame').compute().astype(np.float32)

# Create the maximum projection image
img_max_proj_time = hv.Image(
    max_proj_time, ['width', 'height'], label='Max Over Time'
).opts(
    cmap='magma',
)

# Opacity slider for the overlay
opacity_slider = pn.widgets.FloatSlider(
    start=0, end=1, step=0.1, value=0.3, name='Opacity', align='center', sizing_mode='stretch_width'
)

# Link the slider value to the overlay's alpha (opacity)
opacity_slider.jslink(img_max_proj_time, value='glyph.global_alpha')

### Side Views

<img 
    src="./assets/side_views.png"  
    alt="Static Preview Intermediate App Side Views" 
    align="right" 
    width="70%">

**Here's a static snapshot of the side views with a linked frame indicator lines that we'll make now. 👉**

<div style="clear: both;"></div>

To provide context over the time dimension, we'll add side views showing the mean intensity over time along the height and width dimensions.

The right-side view (as if looking at our 3D volume from the right side) will be a frame-by-height view, and the top-side view will be a width-by-frame view. Using `.persist()` (see [relevant Xarray docs](https://docs.xarray.dev/en/stable/user-guide/dask.html#using-dask-with-xarray) allows us to cache the results of the mean calculations, reducing recomputation and improving performance. `Rasterizing` these views helps to limit the amount of data sent to the browser, ensuring efficient rendering (see [relevant HoloView's docs](https://holoviews.org/user_guide/Large_Data.html)). We will also set `axiswise=True` to prevent the side views from adjusting to the range of the main view, as we want them to be stable references of the full range on their respective axes.

In [ ]:
# Options for the side views
side_view_opts = dict(
    cmap='greys_r',
    tools=['crosshair', 'hover'],
    axiswise=True,
    apply_hard_bounds=True,
    colorbar=False,
    toolbar=None,
)

side_view_width = 175

# Top view: mean over height
top_data = da.mean('height').persist()
top_view = rasterize(
    hv.Image(top_data, kdims=['width', 'frame']).opts(
        frame_height=side_view_width,
        frame_width=da.sizes['width'],
        title='Top Side View',
        xaxis='top',
        **side_view_opts
    )
)

# Right view: mean over width
right_data = da.mean('width').persist()
right_view = rasterize(
    hv.Image(right_data, kdims=['frame', 'height']).opts(
        frame_height=da.sizes['height'],
        title='Right Side View',
        yaxis='right',
        frame_width=side_view_width,
        **side_view_opts
    )
)

We'll add indicators on the side views to show the current frame and the zoom range of the main view.

First, the position of these lines indicates the current frame and is linked to the frame_player value. Instead of using throttled updates for the frame indicator lines, we `bind` directly to the unthrottled value of the frame player since this is a computationally inexpensive operation. This decision ensures that the frame indicators follow the slider in real time, providing a smooth and responsive user experience as the user scrubs through the frames. 

Second, the extents of the indicator lines adjust dynamically as the user interacts with the range (zoom, pan) in the main plot. To achieve this, we use a `streams.RangeXY` from HoloViews, which allows us to subscribe the indicator line extents to the range of the main view plot.

In [ ]:
def plot_hline(frame, x_range, y_range):
    if x_range is None:
        x_range = [int(da.width[0].values), int(da.width[-1].values)]
    return hv.Segments((x_range[0], frame, x_range[1], frame)).opts(axiswise=True)

def plot_vline(frame, x_range, y_range):
    if y_range is None:
        y_range = [int(da.height[0].values), int(da.height[-1].values)]
    return hv.Segments((frame, y_range[0], frame, y_range[1])).opts(axiswise=True)

line_opts = dict(color='red', line_width=3, line_alpha=0.4, line_dash='dashed')
xyrange_stream = hv.streams.RangeXY(source=main_view)
dmap_hline = hv.DynamicMap(pn.bind(plot_hline, frame_player), streams=[xyrange_stream]).opts(**line_opts, **side_view_opts)
dmap_vline = hv.DynamicMap(pn.bind(plot_vline, frame_player), streams=[xyrange_stream]).opts(**line_opts, **side_view_opts)

# Overlay the frame indicators on the side views
top_view_overlay = (top_view * dmap_hline).opts(axiswise=True)
right_view_overlay = (right_view * dmap_vline).opts(axiswise=True)

### Layout


Now we'll assemble all components into a cohesive layout.


In [ ]:
# Overlay the maximum projection on the main view
main_view_overlay = main_view * img_max_proj_time

# Arrange the main view and right side view horizontally
main_and_right_layout = pn.Row(main_view_overlay, right_view_overlay)

# Wrap the player and slider in collapsble Card widgets
player_layout = pn.Card(
    frame_player,
    title='Playback',
    sizing_mode='stretch_width',
    margin=(0, 0, 20, 0),
)

opacity_slider_layout = pn.Card(
    opacity_slider,
    title='Max Projection Overlay',
    sizing_mode='stretch_width',
    margin=(0, 0, 20, 0),
)

# Combine the controls
controls = pn.Column(
    player_layout,
    opacity_slider_layout,
    align='center',
    width=350,
)

# Assemble the full application layout
intermediate_app = pn.Row(
    controls,
    pn.Column(
        top_view_overlay,
        main_and_right_layout,
    )
)

In [ ]:
# intermediate_app

<img 
    src="./assets/intermediate_app.png" 
    alt="Static Preview Intermediate App" 
    align="right" 
    width="70%">

**Here's a static snapshot of what the previous cell produces - the intermediate app with side views and enhanced controls. 👉**

<div style="clear: both;"></div>

## App V3: Advanced Viewer with Annotations and Timeseries


So far, our app allows us to navigate the image stack, but this is usually just the very start of the neuroscience investigation. From this imaging data, it's often essential to next derive a collection of timeseries traces that reflect when individual biological neurons are more or less active, as indicated by the change in image intensity within the spatial regions occupied by each neuron. 

Although there are automated approaches to aid in estimating the spatial outline of each cell, let's work out a complementary approach of how to manually specify and log particular spatial regions of interest using `HoloNote`. Linked to these annotated regions of interest, we can also set up a view that populates with the mean image intensity per frame to form the estimated neural activity fluctuations over time.


### Setting Up the Annotator

Now we'll configure the Annotator object from HoloNote, which manages the annotations and integrates them with our plots.

In this demo, we're using a `connector` to an in-memory SQLite database by specifying `SQLiteDB(filename=':memory:')` for temporary storage of annotations:

In [ ]:
from holonote.annotate import Annotator
from holonote.app import PanelWidgets, AnnotatorTable
from holonote.annotate.connector import SQLiteDB

# Initialize the annotator for 'height' and 'width' dimensions
annotator = Annotator(
    {'height': float, 'width': float},
    fields=['type'],  # Additional field to categorize annotations
    groupby='type',   # Group annotations to enable color-coding and batch actions
    connector=SQLiteDB(filename=':memory:'),
)

**Note on Persistent Storage:** If we don't specify a connector, HoloNote will by default create a persistent SQLite database file named 'annotations.db' in the current directory. You can also specify a custom filename in place of `':memory:'` to store annotations persistently in a specific file, such as `SQLiteDB(filename='my_annotations.db')`.

When working in a real workflow, you'll likely want persistent storage to save your annotations between sessions. Be aware that if a file with the specified name already exists, HoloNote will use it, and any changes committed through the UI will modify this file. This allows you to maintain and update your annotations across different sessions, ensuring your work is saved and accessible later.

Now we can optionally choose to add colors and styling to particular annotation types. For instance, maybe we want all of 'A'-labeled boxes to be red.

In [ ]:
# OPTIONAL: Define colors for annotation 'types' that we might expect to make
color_dim = hv.dim('type').categorize(
    categories={
        'A': 'red',
        'B': 'orange',
        'C': 'cyan',
    },
    default='grey',
)

# Style the annotations
annotator.style.color = color_dim # apply custom colors if we made them
annotator.style.alpha = 0.3

### Configuring Annotation Widgets
To enable interaction with the annotations, we'll create widgets that allow users to view and manage them.

In [ ]:
# Create annotation widgets
panel_widgets = PanelWidgets(annotator)
table_widget = AnnotatorTable(
    annotator,
    tabulator_kwargs={
        'sizing_mode': 'stretch_width',
        'theme': 'midnight',
        'layout': 'fit_columns',
        'sortable': False,
        'stylesheets': [':host .tabulator {font-size: 9px;}'],
    },
)

The above code creates:
- Annotation controls (PanelWidgets) for adding, editing, or deleting annotations. Since we grouped the annotations, `PanelWidgets` will also include a 'Visible' widget to toggle visibility by group.
- An `AnnotatorTable` to display annotations in a tabular format.

We also customize the table's appearance and functionality using `tabulator_kwargs`, adjusting pagination, sizing, layout, theme, style, and text alignment. Check our the [Panel Tabulator](https://panel.holoviz.org/reference/widgets/Tabulator.html) docs for more options.

### Timeseries of Annotated Regions

We'll create timeseries plots that show the mean intensity over time for each annotated region. We will connect the `.on_event` method of the `Annotator` to a `plot_ts` function that returns a subcoordinate timeseries plot of all the mean-aggregated spatial annotations per frame. We'll also add a vertical line to indicate the current frame, which we'll `bind` to our `frame_player` widget for updates.


In [ ]:
# Options for the timeseries plot
curve_opts = dict(
    height=300,
    width=da.sizes['width'] + side_view_width + 225, # align to main_layout width
    show_legend=False,
    xlabel='Frame',
    tools=['hover'],
    line_alpha=0.5,
    framewise=True,
    axiswise=True,
)

# Options for the timeseries' frame indicator
vline_opts = dict(color='grey', line_width=4, alpha=0.5)

# Function to plot timeseries when annotations change
def plot_ts(event):
    curves = {}
    df = annotator.df
    for idx, row in df.iterrows():
        h1, h2, w1, w2 = row[['start[height]', 'end[height]', 'start[width]', 'end[width]']]
        da_sel = da.sel(height=slice(h1, h2), width=slice(w1, w2))
        mean_ts = da_sel.mean(['height', 'width'])
        group = f'G_{row["type"]}'
        label = f'L_{idx[:6]}'
        curve = hv.Curve(mean_ts, group=group, label=label)
        curve = curve.opts(
            subcoordinate_y=True,
            color=panel_widgets.colormap[row['type']],
            **curve_opts,
        )
        curves[(group, label)] = curve
    time_series.object = (vline * hv.Overlay(curves, kdims=['curve'])).opts(
        hv.opts.Curve(xlim=(frames[0], frames[-1])),
    )

# Function to create a vertical line indicating the current frame
def plot_frame_indicator_line(value):
    if value:
        return hv.VSpans((value, value)).opts(
            axiswise=True, framewise=True, **vline_opts
        )

# Get the range of frames
frames = da.coords['frame'].values

# Create a DynamicMap for the frame indicator line
vline = hv.DynamicMap(pn.bind(plot_frame_indicator_line, frame_player)).opts(
    hv.opts.VLine(**vline_opts)
)

# Initialize the timeseries pane
time_series = pn.pane.HoloViews(
    vline * hv.Curve([]).opts(
        xlim=(frames[0], frames[-1]),
        title='Create an annotation in the image',
        **curve_opts,
    )
)

# Connect the annotation events to the plotting function
annotator.on_event(plot_ts)

### Defining Annotations

Optionally, if we have some pre-existing annotations, we can add to the annotation database. If you started without any existing annotations, you can skip this step, and the Annotator will start without any entries, ready for you to add new ones interactively.


In [ ]:
annotations_df = pd.DataFrame(
    [[430, 480, 80, 130, 'A']],
    columns=['x1', 'x2', 'y1', 'y2', 'type']
)
annotations_df

In [ ]:
annotator.define_annotations(
    annotations_df,
    width=("x1", "x2"),
    height=("y1", "y2"),
    type='type',
)

### Updating the Layout

We'll update the application layout from the intermediate app version to include the `annotator`, along with the associated widgets and timeseries plot.


In [ ]:
# Overlay the annotator on the main view
main_view_overlay_anno = (main_view_overlay * annotator).opts(axiswise=True)
# Overlay the annotator on the side views
top_view_overlay_anno = (top_view_overlay * annotator).opts(axiswise=True)
right_view_overlay_anno = (right_view_overlay * annotator).opts(axiswise=True)

# Combine annotation controls and table into a Card
annotator_widgets = pn.Card(
    pn.Column(panel_widgets, table_widget),
    title='Annotator',
    sizing_mode='stretch_width',
    margin=(0, 0, 20, 0),
    collapsed=False,
)

# Add the annotator widgets to the controls
controls.append(annotator_widgets)

# Create the main content layout
main_layout_anno = pn.Column(
    top_view_overlay_anno,
    pn.Row(main_view_overlay_anno, right_view_overlay_anno),
    time_series,
)

# Assemble the final application layout
advanced_app = pn.Row(
    controls,      # Controls on the left
    main_layout_anno,   # Main content on the right
    align='start',
)

### Working with Annotations

This [video preview](https://www.youtube.com/embed/Qa-wrIdMYH0?si=KDzApOEt2e4ROu-l) is a great way to see how an annotation is created with the HoloNote controls.

How to add new annotations: 
1. Ensure that 'Box select (x-axis)' is selected in the Bokeh Toolbar on the right of the main frame plot.
2. Ensure that the '+' button is selected in the Annotation widgets on the very left.
3. Select a spatial range by clicking and dragging on the main frame plot.
4. Enter the name of the group that you want the annotation to be added to in the 'type' text box.
5. Edit or delete annotations using the provided widgets and table.
6. If you are using a persistent file on disk for the annotations and are ready to commit, hit the triangle button in the Annotation widgets to commit changes to the file.

<img  
    src="./assets/advanced_app_nb_steps.png" 
    alt="Static Preview Advanced App in Notebook" 
    align="right" 
    width="80%">

**Here's a static snapshot of what the advanced app looks like in the notebook, after creating a box in the main view and assigning it a type in the annotation widget. 👉**

<div style="clear: both;"></div>

In [ ]:
advanced_app

## Standalone App Extension

HoloViz Panel allows for the deployment of this complex visualization as a standalone, template-styled, interactive web application (outside of a Jupyter Notebook). Read more about Panel [here](https://panel.holoviz.org/).

We'll add our plots to the `main` area of a Panel Template component and the widget-controls to the `sidebar`. Finally, we'll set the entire component to be `servable`.

To launch the standalone app, activate the same conda environment and run `panel serve <path-to-this-file> --show` in the command line to open the application in a browser window (tip: use the `--dev` flag to auto update the app when the file changes).

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Warning</p>
    It is not recommended to have both a notebook version of the app and the served version of the same application running simultaneously. Prior to serving the standalone application, clear the notebook output, restart the notebook kernel, and save the unexecuted notebook file.
</div>

In [ ]:
servable_app = pn.template.FastListTemplate(
    sidebar = controls,
    title = "HoloViz + Bokeh Time-Lapse Microscopy with Annotation",
    main = main_layout_anno,
    main_layout=None,
    sidebar_width=350,
    theme="dark",
    accent="#30023f"
).servable()

## Copyable V3 App Code

<details><summary> Expand to reveal </summary>

```python
print('TODO once the the notebook is reviewed')
```
    
</details>

## Next Steps

- Experiment with your own datasets, extending the applications with additional features as needed.

---

## Resources

| What? | Why? |
| --- | --- |
| [Minian Repository](https://github.com/denisecailab/minian) | Analysis pipeline and visualization tool for Miniscope data |
| [Miniscope Wiki](http://miniscope.org/index.php/Main_Page) | Further context for the demo application |
| [HoloNote](https://holonote.holoviz.org/) | HoloNote package for annotation capabilities in HoloViz applications |
